In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [18]:
df_cust_data = pd.read_excel('Customer_Data.xlsx')
df_plant = pd.read_excel('Plant Master.xlsx')
df_invoice = pd.read_csv('Final_invoice.csv',error_bad_lines=False, index_col=False, dtype='unicode')

In [19]:

remove_cols = []
for column in df_invoice.columns:
    #print("Percentage of missing values in {} is: {}".format(column, (df_invoice[column].isnull().sum()*100)/len(df_invoice)))
    if ((df_invoice[column].isnull().sum()*100)/len(df_invoice)) > 10:
        remove_cols.append(column)
        
#drop columns where data missing less then 90 %
df_invoice.drop(columns=remove_cols, axis = 1,inplace=True)
#Remove punctution char , space from column name 
df_invoice.columns = df_invoice.columns.str.replace(r'[^\w\s]', '')
df_invoice.columns = df_invoice.columns.str.replace(' ', '')
#convert column name to upper case
df_invoice.columns = df_invoice.columns.str.upper()

In [4]:
df_invoice.columns

Index(['UNNAMED0', 'AREALOCALITY', 'CITY', 'CUSTTYPE', 'CUSTOMERNO',
       'DISTRICT', 'GATEPASSTIME', 'INVOICEDATE', 'INVOICENO', 'INVOICETIME',
       'JOBCARDNO', 'JOBCARDDATE', 'JOBCARDTIME', 'KMSREADING', 'LABOURTOTAL',
       'MAKE', 'MISCTOTAL', 'MODEL', 'OSLTOTAL', 'ORDERTYPE', 'PARTSTOTAL',
       'PINCODE', 'PLANT', 'PLANTNAME1', 'PRINTSTATUS', 'RECOVRBLEXP',
       'REGNNO', 'TOTALAMTWTDTAX', 'USERID'],
      dtype='object')

In [49]:
df_invoice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440143 entries, 0 to 492313
Data columns (total 32 columns):
UNNAMED0          440143 non-null object
AREALOCALITY      417330 non-null object
CITY              440143 non-null object
CUSTTYPE          440143 non-null object
CUSTOMERNO        440143 non-null object
DISTRICT          440143 non-null object
GATEPASSTIME      440143 non-null object
INVOICEDATE       440143 non-null object
INVOICENO         440143 non-null object
INVOICETIME       440143 non-null object
JOBCARDNO         440143 non-null object
JOBCARDDATE       440143 non-null object
JOBCARDTIME       440143 non-null object
KMSREADING        440143 non-null object
LABOURTOTAL       440143 non-null float64
MAKE              440143 non-null object
MISCTOTAL         440143 non-null float64
MODEL             438830 non-null object
OSLTOTAL          440143 non-null float64
ORDERTYPE         440143 non-null object
PARTSTOTAL        440143 non-null float64
PINCODE           440143

In [20]:
num_columns = ['LABOURTOTAL','MISCTOTAL','OSLTOTAL','PARTSTOTAL','TOTALAMTWTDTAX']
for col in num_columns:
    #replace null with zer
    df_invoice[col].fillna(0)
    #Convert to float & then Int
    df_invoice[col] = df_invoice[col].astype(float)
    #df_invoice[col] = df_invoice[col].astype(int)

df_invoice['UNNAMED0'] = df_invoice['UNNAMED0'].astype(int)

In [21]:
#remove rows where total invoice amount is null
df_invoice.drop(index=df_invoice[df_invoice['TOTALAMTWTDTAX'] == 0].index,inplace=True)
#remove rows where total amount less than zero
df_invoice.drop(index=df_invoice[df_invoice['TOTALAMTWTDTAX'] < 0].index,inplace=True)
#remove rows where Labour amount less than zero
df_invoice.drop(index=df_invoice[df_invoice['LABOURTOTAL'] < 0].index,inplace=True)

In [15]:
df_invoice.shape

(369814, 31)

In [23]:

df_invoice = df_invoice.join(df_pincodes.set_index(['Pincode','PO_StateName']), on=['PINCODE','DISTRICT'])

In [24]:
df_invalid_state = df_invoice[df_invoice['DISTRICT'] != df_invoice['StateName']].copy()
df_invalid_state['PO_District'] = df_invalid_state['CITY']
df_invalid_state['StateName'] = df_invalid_state['DISTRICT']
df_invalid_city = df_invalid_state[['CITY','PLANT']][df_invalid_state['CITY'].str.len() < 3].copy()
df_invalid_city = df_invalid_city.join(df_plant.set_index('Plant'), on='PLANT')

for index, row in df_invalid_city.iterrows(): 
    df_invalid_state.loc[index,'PO_District'] = row['City']

In [10]:
df_invoice.shape

(440138, 31)

In [25]:

df_invoice.drop(index=df_invalid_state.index,inplace=True)

df_final= pd.concat([df_invoice,df_invalid_state])

df_final.sort_values(by='UNNAMED0',inplace=True)

df_final.drop(columns=['CITY','DISTRICT'],axis=1,inplace=True)
df_final.rename(columns={"PO_District": "CITY", "StateName": "STATE"},inplace=True)
df_final['CITY'] = df_final['CITY'].str.upper()
df_final['STATE'] = df_final['STATE'].str.upper()

In [13]:
df_final.shape

(440138, 29)

In [16]:
df_final.columns

Index(['UNNAMED0', 'AREALOCALITY', 'CUSTTYPE', 'CUSTOMERNO', 'GATEPASSTIME',
       'INVOICEDATE', 'INVOICENO', 'INVOICETIME', 'JOBCARDNO', 'JOBCARDDATE',
       'JOBCARDTIME', 'KMSREADING', 'LABOURTOTAL', 'MAKE', 'MISCTOTAL',
       'MODEL', 'OSLTOTAL', 'ORDERTYPE', 'PARTSTOTAL', 'PINCODE', 'PLANT',
       'PLANTNAME1', 'PRINTSTATUS', 'RECOVRBLEXP', 'REGNNO', 'TOTALAMTWTDTAX',
       'USERID', 'CITY', 'STATE', 'JOBCARDDATETIME', 'INVOICEDATETIME',
       'SERVICETIME'],
      dtype='object')

In [61]:
res = df_final.join(df_mh_group.set_index(['CUSTOMERNO','ORDERTYPE']), on=['CUSTOMERNO','ORDERTYPE'])
res = res[res['STATE'] == 'MAHARASHTRA']
res = res[res['INVOICEDATE'] > '2016-01-01']

In [63]:
res.pivot_table(index=['CUSTTYPE'],columns=['ORDERTYPE'],values='cust_lifetime_value',aggfunc='count',fill_value=0).reset_index()

ORDERTYPE,CUSTTYPE,Accidental,Mechanical,Paid Service,Running Repairs,SMC Value Package
0,Corporate others,0,0,1,1,0
1,Corporate- M&M,0,1,15,39,0
2,Fleets,0,3,3,5,0
3,Franchise,0,0,0,2,0
4,MFCWL,0,1,3,2,0
5,Retail,0,2886,12273,14444,339


In [62]:
res

,UNNAMED0,AREALOCALITY,CUSTTYPE,CUSTOMERNO,GATEPASSTIME,INVOICEDATE,INVOICENO,INVOICETIME,JOBCARDNO,JOBCARDDATE,JOBCARDTIME,KMSREADING,LABOURTOTAL,MAKE,MISCTOTAL,MODEL,OSLTOTAL,ORDERTYPE,PARTSTOTAL,PINCODE,PLANT,PLANTNAME1,PRINTSTATUS,RECOVRBLEXP,REGNNO,TOTALAMTWTDTAX,USERID,CITY,STATE,JOBCARDDATETIME,INVOICEDATETIME,SERVICETIME,num_days,num_transactions,spent_money,avg_spend_value,profit_margin,CLV,cust_lifetime_value
295028,730169,MUMBAI,Retail,119371,15:41:33,2016-01-02,7005202789,11:27:27,300412,2016-01-02,10:52:24,35654,1125.76,MARUTI SUZUKI,291.45,ZEN ESTILO,0.00,SMC Value Package,2082.07,400089,BC01,THANE,YES,0.0,MH02BD6668,3499.28,BC01SA2,MUMBAI,MAHARASHTRA,2016-01-02 10:52:24,2016-01-02 11:27:27,0.0,0.0,1.0,3499.28,3499.280000,174.9640,7161.757172,1253049.0
295029,730170,THNAE,Retail,128929,13:05:33,2016-01-02,7005202790,13:03:16,300447,2016-01-02,11:50:58,70730,233.85,MARUTI SUZUKI,0.00,SWIFT,0.00,Running Repairs,195.44,400604,BC01,THANE,YES,0.0,MH12EM2307,429.29,BC01SA2,THANE,MAHARASHTRA,2016-01-02 11:50:58,2016-01-02 13:03:16,0.0,89.0,2.0,4322.10,2161.050000,216.1050,4422.885661,955807.0
295031,730172,NaN,Retail,E10724,16:12:42,2016-01-02,7005202792,15:57:28,300505,2016-01-02,12:20:38,50220,69.97,FIAT,0.00,PALIO,0.00,Running Repairs,297.65,400606,BC01,THANE,YES,0.0,MH04CT2685,367.62,BC01SA2,THANE,MAHARASHTRA,2016-01-02 12:20:38,2016-01-02 15:57:28,0.0,1180.0,2.0,1635.78,817.890000,81.7890,1673.924228,136908.0
295032,730173,THNAE,Retail,92107,16:28:09,2016-01-02,7005202793,16:23:22,300561,2016-01-02,13:06:01,79061,698.64,MARUTI SUZUKI,0.00,SWIFT,0.00,Paid Service,44.44,400603,BC01,THANE,YES,0.0,MH04GE5258,743.08,BC01SA2,THANE,MAHARASHTRA,2016-01-02 13:06:01,2016-01-02 16:23:22,0.0,115.0,2.0,3420.47,1710.235000,171.0235,3500.230841,598621.0
295033,730174,HIRANANDANI ESTATE,Retail,65389,17:08:28,2016-01-02,7005202794,17:02:22,300676,2016-01-02,15:38:45,83195,426.03,MAHINDRA & MAHINDRA,0.00,LOGAN P,0.00,Running Repairs,2089.08,400610,BC01,THANE,YES,0.0,MH43R8997,2515.11,BC01SA2,THANE,MAHARASHTRA,2016-01-02 15:38:45,2016-01-02 17:02:22,0.0,196.0,3.0,17164.82,5721.606667,858.2410,11710.053948,10050048.0
295034,730175,THANE,Retail,35473,10:06:17,2016-01-02,7005202795,18:33:51,300701,2016-01-02,15:53:46,64975,65.02,MARUTI SUZUKI,0.00,SWIFT,0.00,Running Repairs,826.31,400607,BC01,THANE,YES,0.0,MH06AW9047,891.33,BC01SA2,THANE,MAHARASHTRA,2016-01-02 15:53:46,2016-01-02 18:33:51,0.0,755.0,6.0,9298.33,1549.721667,464.9165,3171.718257,1474584.0
295036,730177,THANE,Retail,76158,11:11:57,2016-01-03,7005202797,11:08:56,300563,2016-01-02,13:08:31,62246,999.13,MARUTI SUZUKI,0.00,SWIFT,0.00,Paid Service,42.68,600406,BC01,THANE,YES,0.0,MH43N9318,1041.81,BC01SA2,THNAE,MAHARASHTRA,NaT,NaT,NaN,NaN,1.0,1041.81,1041.810000,52.0905,2132.207265,111067.0
295037,730178,THANE,Retail,16228,12:15:12,2016-01-03,7005202798,12:10:21,300805,2016-01-03,12:02:29,43826,0.00,TATA MOTORS,262.01,INDICA,0.00,Running Repairs,0.00,400606,BC01,THANE,YES,0.0,MH04CT612,262.01,BC01WM1,THANE,MAHARASHTRA,2016-01-03 12:02:29,2016-01-03 12:10:21,0.0,800.0,2.0,1577.37,788.685000,78.8685,1614.152184,127305.0
295038,730179,THANE,Retail,18298,12:22:38,2016-01-03,7005202799,12:18:53,300607,2016-01-02,14:29:50,25931,999.14,HYUNDAI,0.00,I20,0.00,Paid Service,0.00,400607,BC01,THANE,YES,0.0,MH02BP1353,999.14,BC01SA2,THANE,MAHARASHTRA,2016-01-02 14:29:50,2016-01-03 12:18:53,0.0,0.0,1.0,999.14,999.140000,49.9570,2044.877249,102155.0
295039,730180,THANE,Retail,6776,12:32:21,2016-01-03,7005202800,12:24:27,300672,2016-01-02,15:24:52,105104,210.04,FORD,0.00,FIESTA,850.16,Running Repairs,448.10,400606,BC01,THANE,YES,0.0,MH04CT4993,1508.30,BC01SA2,THANE,MAHARASHTRA,2016-01-02 15:24:52,2016-01-03 12:24:27,0.0,1135.0,4.0,10048.37,2512.092500,502.4185,5141.342356,2583105.0


In [210]:
#Regression model for year wise sale  
df_mh['INVOICEDATE'].dropna(inplace=True)
df_mh['YEAR'] = df_mh['INVOICEDATE'].apply(lambda x: x.strftime('%Y'))
sale=df_mh.pivot_table(index=['CUSTOMERNO'],columns=['YEAR'],values='TOTALAMTWTDTAX',aggfunc='sum',fill_value=0).reset_index()

In [216]:
sale.head()

YEAR,CUSTOMERNO,2012,2013,2014,2015,2016,CLV
0,10,6161.89,4943.67,0.00,0.00,0.0,4943.67
1,100,20799.74,0.00,0.00,0.00,0.0,0.00
2,1000,2817.57,2730.26,3265.61,0.00,0.0,5995.87
3,100001,0.00,0.00,0.00,4263.75,0.0,4263.75
4,100008,0.00,0.00,0.00,2079.81,0.0,2079.81


In [214]:
sale['CLV']=sale.iloc[:,2:].sum(axis=1)

X=sale[['2012','2013','2014','2015','2016']]
y=sale[['CLV']]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

from sklearn.linear_model import LinearRegression

# instantiate linearreg 
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions 
y_pred = linreg.predict(X_test)

In [220]:
print(linreg.intercept_)
print(linreg.coef_)

[-4.72937245e-11]
[[1.50869701e-13 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00]]
